In [1]:
import praw
from psaw import PushshiftAPI

import datetime as dt

import pandas as pd
import numpy as np
from IPython.display import display, Markdown, Latex

from os import listdir
from os.path import isfile, join
import csv

from tqdm.notebook import tqdm,tnrange
import pickle

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

import matplotlib.pyplot as plt
import seaborn as sns

from wordcloud import WordCloud, STOPWORDS

In [3]:
import import_ipynb
from reddit_access import client_id, client_secret, password, user_agent, username

#PSAW TO RETRIEVE IDs OF SUBMISSIONS
api = PushshiftAPI()

#PRAW TO RETRIEVE ACTUAL CONVERSATIONS
reddit = praw.Reddit(client_id=client_id, 
                     client_secret=client_secret,
                     password=password, 
                     user_agent=user_agent,
                     username=username)

reddit.user.me()

Redditor(name='micheledinelli')

In [23]:
mypath = "./submissions/"
files = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]

merged_file = []
for filename in files:
    with open(filename, 'r') as csv_file:
        file = csv.reader(csv_file)
        for row in file:
            merged_file.append(row)
            
merged_df = pd.DataFrame(merged_file, columns=['index', 'id'])
merged_df.drop(columns=['index'], inplace=True)
merged_df.to_csv('all_ids.csv')

In [24]:
all_ids = pd.read_csv('all_ids.csv')
all_ids.dropna(inplace=True)

In [ ]:
comments = []
for id in tqdm (all_ids['id'], desc="Loading comments"):
    if id != str(0):
        post = reddit.submission(id)
        for comment in post.comments:
            comments.append([comment.parent_id, comment.author, comment.score, comment.id, comment.subreddit, comment.body, comment.created])

Loading comments:   0%|          | 0/78851 [00:00<?, ?it/s]

In [55]:
df_comments = pd.DataFrame(comments, columns=['parent_id', 'author', 'score', 'id', 'subreddit', 'body', 'created'])

df_comments['created'] = df_comments['created'].apply(lambda x : dt.datetime.fromtimestamp(x))

df_comments.to_csv('./comments_scraped/comments.csv')
df_comments.head()

,parent_id,author,score,id,subreddit,body,created
0,t3_k4xgh0,biljardbal,6,gebmqwv,CrohnsDisease,"I tried to make one like this, but it seems th...",2020-12-02 02:32:38
1,t3_k4xgh0,loonalady,6,geboap0,CrohnsDisease,"Confused why people are downvoting, is this of...",2020-12-02 02:47:01
2,t3_k4x6p1,cheetoPalmer,2,gebfaos,CrohnsDisease,"When I’m in the thick of it, the only thing I’...",2020-12-02 01:23:34
3,t3_k4x6p1,forestcall,2,gedv0qx,CrohnsDisease,What works well for me is superfood chocolate ...,2020-12-02 17:34:45
4,t3_k4w6h7,thesch,9,geb54qt,CrohnsDisease,My doc is the opposite problem where when I as...,2020-12-01 23:52:43


In [49]:
df_comments.shape

(24, 7)